In [ ]:
from google.colab.auth import authenticate_user

In [ ]:
authenticate_user()

In [ ]:
path = "gs://nlp_599/data/newfuncom.csv"

In [ ]:
!gsutil cp $path ./newfuncom.csv

Copying gs://nlp_599/data/newfuncom.csv...
\ [1 files][622.8 MiB/622.8 MiB]   54.0 MiB/s                                   
Operation completed over 1 objects/622.8 MiB.                                    


In [ ]:
import pandas as pd

dataset = pd.read_csv("newfuncom.csv")

In [ ]:
dataset

,comment,code_words,doc_id,split_name,tokenized_code
0,mouse up of abstract data view ok button,public void mouse up final mouse event mouseev...,43226887,train,{void} {mouse} {up} {final} {mouse} {event} {m...
1,returns the state constant value of code new s...,public int get state return new,43227058,train,{int} {get} {state} {return} {new}
2,returns the state constant value of code clean...,public int get state return clean,43227061,train,{int} {get} {state} {return} {clean}
3,this method returns the exact sql command like...,public string get sql command return sql,43227066,train,{string} {get} {sql} {command} {return} {sql}
4,this method sets the text of the sql command,public void set sql command final string sql c...,43227068,train,{void} {set} {sql} {command} {final} {string} ...
...,...,...,...,...,...
1415297,test calculation of center position with percent,public void test calculate percent5 my test cl...,14093055,train,{void} {test} {calculate} {percent5} {my} {tes...
1415298,test the constructor,public void test jtree mouse event data7p m ev...,14093057,train,{void} {test} {jtree} {mouse} {event} {data7p}...
1415299,this method will test the enter click and leav...,public void test enter drop down click and lea...,14093058,train,{void} {test} {enter} {drop} {down} {click} {a...
1415300,test the constructor,public void test mouse event data1 m event1 ne...,14093061,train,{void} {test} {mouse} {event} {data1} {m} {eve...


In [ ]:
df = pd.read_csv('docstring_comparison_data.csv')

In [ ]:
df.head(100)

In [ ]:
def clear_special_words(comment):
    comment = comment.replace("&apos;", "")
    return comment
df['generated_comment'] = df['generated_comment'].apply(lambda x: clear_special_words(x))

In [ ]:
df['reference_comment'] = df['reference_comment'].apply(lambda x: clear_special_words(x))

In [ ]:
df

In [ ]:
def delete_after_dcnl(comment):
    words = comment.split(" ")
    cnt = 0
    dcnl = 0
    for word in words:
        if word == "DCNL":
            dcnl = cnt
            break
        cnt += 1
    if dcnl != 0:
        words = words[:dcnl]
    return " ".join(words)

df['reference_comment'] = df['reference_comment'].apply(lambda x: delete_after_dcnl(x))
df['generated_comment'] = df['generated_comment'].apply(lambda x: delete_after_dcnl(x))

In [ ]:
df

,doc_id,generated_comment,reference_comment
0,train19316,Return the number of CPUs currently running on...,Get the number of CPU processes on the current...
1,train31156,Display the info page.,In@@ fo page ( link from main header )
2,train19086,Decorator to decorate a function that returns ...,A decorator that tests that the decorated func...
3,train59453,Return the safe string for use in a shell.,Return a taxon identifier according to N@@ EX@...
4,train61981,Return True if the url is safe to be used. &apos,Return ``True`` if the url is a safe redirecti...
...,...,...,...
14167,train52113,Return the realm for the given entity,Get the default realm ( = the immediate OU anc...
14168,train5439,Remove a pidfile from the system.,Remove the named PID file if it exists .
14169,train33081,Plot a matplotlib figure.,Re@@ plot a matplotlib figure with plotly .
14170,train17305,Return a fresh instance of the hash object.,Return a fresh instance of the hash object .


In [ ]:
from typing import List, Tuple
import nltk

from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

def get_tokens_for_dataset_comment(comment: str) -> List[str]:
    toks = comment.split(" ")
    toks = [t.lower() for t in toks]
    return toks

def get_tokenized_str_for_dataset_comment(comment: str) -> str:
    """Space seperated string of comment tokens"""
    return " ".join(get_tokens_for_dataset_comment(comment))

def _prepare_strings(
    refs: List[str],
    hypotheses: List[str],
    pretokenized: bool = False
) -> Tuple[List[str], List[str]]:
    
    ref_strings = [r.lower() for r in refs]
    hypotheses = [h.lower() for h in hypotheses]

    if not pretokenized:
        ref_strings = [get_tokenized_str_for_dataset_comment(s) for s in ref_strings]
        hypotheses = [get_tokenized_str_for_dataset_comment(s) for s in hypotheses]
    return ref_strings, hypotheses

def eval_bleu_1(
    refs: List[str],
    hypotheses: List[str],
):
    ref_strings, hypotheses = _prepare_strings(refs, hypotheses)
    
    refs = [[r.split()] for r in ref_strings]
    preds = [h.split() for h in hypotheses]

    return nltk.translate.bleu_score.corpus_bleu(refs, preds, weights=(1, 0, 0, 0), smoothing_function=SmoothingFunction().method4)*100

In [ ]:
generated_comments = []
reference_comments = []
for index, row in df.iterrows():
    generated_comments.append(row['generated_comment'])
    reference_comments.append(row['reference_comment'])
eval_bleu_1(reference_comments, generated_comments)

26.37704422274027

In [ ]:
df.head(100)

,doc_id,generated_comment,reference_comment
0,train19316,Return the number of CPUs currently running on...,Get the number of CPU processes on the current...
1,train31156,Display the info page.,In@@ fo page ( link from main header )
2,train19086,Decorator to decorate a function that returns ...,A decorator that tests that the decorated func...
3,train59453,Return the safe string for use in a shell.,Return a taxon identifier according to N@@ EX@...
4,train61981,Return True if the url is safe to be used. &apos,Return ``True`` if the url is a safe redirecti...
...,...,...,...
95,train20607,Install a library.,Spec@@ ify a third-party package to use .
96,train64922,Convert a structure structure to a structure.,Converts the \ blocks \ key from a list &#91...
97,train63915,Decorator for functions that return a function...,decorator to smar@@ tly _ sympify function arg...
98,train54119,Convert a tuple of record values to a timestamp.,Extract timestamp from HBase tuple record .
